## Imports

In [2]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

we first update yesterdays date

In [3]:
yesterday = "2018-09-17"

## Load the data set
First let's load the diffent files of our datase. 

In [4]:
# Load from csv
df_product_hierachy = pd.read_csv('../dataset/turkish_retail_data/product_hierarchy.csv', 
                                 delimiter= ',',
                                 dtype={'product_id':'category',
                                        'cluster_id':'category',
                                        'hierarchy1_id':'category',
                                        'hierarchy2_id':'category',
                                        'hierarchy3_id':'category',
                                        'hierarchy4_id':'category',
                                        'hierarchy5_id':'category'})
df_sales = pd.read_csv('../dataset/turkish_retail_data/sales.csv',
                       delimiter=',', 
                       dtype={"product_id":"category", 
                              "store_id":"category",
                              "promo_type_1":"category",
                              "promo_bin_1":"category",
                              "promo_type_2":"category",
                              "promo_bin_2":"category",
                              "promo_discount_2":"category",
                              "promo_discount_type_2":"category"},
                       parse_dates=["date"])
df_sales = df_sales.join(df_product_hierachy.set_index('product_id'), on='product_id')

## We first build a tool for anomaly detection on daily store performance measured against the 14-day running average:

In [67]:
daily_revenue_by_store = df_sales.groupby(['store_id','date'])['revenue'].agg('sum').reset_index()

Only take into account revenue per store over the last 14 days

In [78]:
daily_revenue_by_store_last_two_weeks = daily_revenue_by_store[daily_revenue_by_store.date.isin(pd.date_range(pd.Timedelta(-13, unit='d')+pd.to_datetime(yesterday, format='%Y-%m-%d', errors='ignore').date(), periods=14))]

We filter out stores that are not regularaly reporting numbers:

In [79]:
daily_revenue_by_store_last_two_weeks.loc[daily_revenue_by_store_last_two_weeks['revenue'].isnull(),'revenue']=0

/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [77]:
daily_revenue_by_store_last_two_weeks['revenue'] = daily_revenue_by_store_last_two_weeks['revenue'].replace(np.nan, 0)

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [54]:
daily_revenue_by_store_last_two_weeks['revenue'] = daily_revenue_by_store_last_two_weeks['revenue'].fillna(0)
filter_good_stores = daily_revenue_by_store_last_two_weeks.groupby(['store_id']).agg(['max', 'min']).reset_index()
filter_good_stores.columns = list(map(''.join, filter_good_stores.columns.values))
filter_good_stores = filter_good_stores.loc[filter_good_stores.revenuemax != filter_good_stores.revenuemin]
good_stores = filter_good_stores['store_id'].tolist()
daily_revenue_by_store_last_two_weeks = daily_revenue_by_store_last_two_weeks[daily_revenue_by_store_last_two_weeks.store_id.isin(good_stores)]

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


We now create a dataframe that shows average sales over the past two weeks per store and yesterday's revenue per store.

In [55]:
two_week_store_mean = daily_revenue_by_store_last_two_weeks.groupby(['store_id']).agg('mean').reset_index()
two_week_store_mean = two_week_store_mean.dropna()
two_week_store_mean = two_week_store_mean.rename(columns={"revenue": "mean_revenue"})

yesterday_store_revenue = daily_revenue_by_store_last_two_weeks.loc[daily_revenue_by_store_last_two_weeks.date == yesterday]
yesterday_store_revenue = yesterday_store_revenue.drop(columns=['date'])
yesterday_store_revenue = yesterday_store_revenue.rename(columns={"revenue": "yesterday_revenue"})

store_mean_yesterday = two_week_store_mean.set_index('store_id').join(yesterday_store_revenue.set_index('store_id'))

We now isolate stores whose revenue yesterday was less than 5% of the moving 14-day average:

In [56]:
underperforming_stores_df = store_mean_yesterday.loc[store_mean_yesterday.yesterday_revenue < 0.05 * store_mean_yesterday.mean_revenue].reset_index()
underperforming_stores = underperforming_stores_df['store_id'].tolist()

In [57]:
underperforming_stores

['S0017', 'S0136']

In [66]:
result = {}
for store in underperforming_stores:
    temp = daily_revenue_by_store_last_two_weeks[daily_revenue_by_store_last_two_weeks.store_id.isin([store])]
    result[store] = {}
    result[store]["chart_type"] = 0
    result[store]["data"] = temp[['date', 'revenue']].to_numpy()
    result[store]["text1"] = f"Yesterday's daily revenue for store {store} fellow below 5 percent of the 14 day average."
    result[store]["text2"] = "Check in with store manager to determine cause."
result

{'S0017': {'chart_type': 0,
  'data': array([[Timestamp('2018-09-04 00:00:00'), 134.74],
         [Timestamp('2018-09-05 00:00:00'), 207.39000000000004],
         [Timestamp('2018-09-06 00:00:00'), 275.56000000000006],
         [Timestamp('2018-09-07 00:00:00'), 106.89000000000001],
         [Timestamp('2018-09-08 00:00:00'), 176.14],
         [Timestamp('2018-09-09 00:00:00'), 162.75],
         [Timestamp('2018-09-10 00:00:00'), 46.32],
         [Timestamp('2018-09-11 00:00:00'), 452.76],
         [Timestamp('2018-09-12 00:00:00'), 287.28999999999996],
         [Timestamp('2018-09-13 00:00:00'), 87.05],
         [Timestamp('2018-09-14 00:00:00'), 174.73],
         [Timestamp('2018-09-15 00:00:00'), 65.6],
         [Timestamp('2018-09-16 00:00:00'), 119.83],
         [Timestamp('2018-09-17 00:00:00'), 0.0]], dtype=object),
  'text1': "Yesterday's daily revenue for store S0017 fellow below 5 percent of the 14 day average.",
  'text2': 'Check in with store manager to determine cause.'},
